In [1]:
%%time

%pip install -U tornado
%pip install "dask[complete]"

import warnings
warnings.filterwarnings("ignore")
import shutil
import os
import pandas as pd
import matplotlib
matplotlib.use(u'nbAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import pickle
from sklearn import preprocessing
import pandas as pd
import random as r
from sklearn.svm import SVC  # Import the Support Vector Machine classifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.7/427.7 KB 2.0 MB/s eta 0:00:001.7 MB/s eta 0:00:01
  Attempting uninstall: tornado
    Found existing installation: tornado 6.3.2
    Uninstalling tornado-6.3.2:
      Successfully uninstalled tornado-6.3.2
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 1.6 MB/s eta 0:00:003.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 KB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.3 MB/s eta 0:00:005.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Defining the root directory where your .asm files are stored
root_directory = os.path.expanduser("~/Documents/extracted_files/test")  # Update the path as needed

# Initialize a list to store .asm file paths and their contents
asm_files = []

# os.walk --> Recursively search for .asm files in the subdirectories within subdirectories
for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith(".asm"):
            asm_path = os.path.join(dirpath, filename)
            with open(asm_path, 'r', encoding='utf-8', errors='ignore') as file:
                asm_content = file.read()
                asm_files.append((asm_path, asm_content))

